In [5]:
import pandas as pd
import numpy as np
import sqlalchemy as _sql
import sqlalchemy.ext.declarative as _declarative
import sqlalchemy.orm as _orm

In [6]:
# TODO: change to real database
# DATABASE_URL = "postgresql+psycopg2://db_user:password@db:5432/inzynierka_db"
# engine = _sql.create_engine(DATABASE_URL)
# SessionLocal = _orm.sessionmaker(autocommit=False, autoflush=False, bind=engine)
# Base = _declarative.declarative_base()

/tmp/ipykernel_12/2114804613.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = _declarative.declarative_base()


# Data reading

In [12]:
class ReadData:
    
    def __init__(self, path):
        
        self.path = path    # to be replaced by database connection
        self.data = None
        
    def read_data(self):
        
        columns = ["user_id", "game_name", "purchase_play", "hours_played", "0"] # for now as there are no headers
        self.data = pd.read_csv(self.path, names=columns, header=None)
        
        return self.data

path = "data/steam-200k.csv"
rd = ReadData(path)
data = rd.read_data()
data


,user_id,game_name,purchase_play,hours_played,0
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0
...,...,...,...,...,...
199995,128470551,Titan Souls,play,1.5,0
199996,128470551,Grand Theft Auto Vice City,purchase,1.0,0
199997,128470551,Grand Theft Auto Vice City,play,1.5,0
199998,128470551,RUSH,purchase,1.0,0


# Block building
### Rozumiem, że suffix table to tu średnio, wiec głównie na początku tumbling window, potem można  hopping window, jakieś inne też bloki budowane na podstawie sąsiedztwa - no to już do dorobienia co tam się wymyśli

In [119]:
class BlockBuilding:
    
    def __init__(self, data, sort_flag=True):
        self.data = data
        self.blocks = []
        self.method = None
        self.sort_flag = sort_flag
        self.num_blocks = 1
        
    def set_method(self, method):
        if method in ['suffix_array', 'tumbling_window']:
            self.method = method
        else:
            raise ValueError("Method not supported")
        pass
        
    def sort_data(self, columns):
        
        if self.sort_flag:
            if not columns:
                raise ValueError("You must specify the columns to sort by.")
            else:
                self.data = self.data.sort_values(by=columns)
        
        return self.data
        
    
    def display_block(self, block_index=1):
        
        if self.blocks is None:
            raise ValueError("No blocks have been generated. Run any of the block building methods first.")
        
        return self.blocks[self.blocks['block_id'] == block_index]
    
    
    def get_blocks(self):
        
        if self.blocks is None:
            raise ValueError("No blocks have been generated. Run any of the block building methods first.")
        
        return self.blocks    
    # # Suffix array method - maybe in future
    # def suffix_array(self):
    #     columns = self.data.columns
    #     suffix_dict = {}
    #     
    #     for column in columns:
    #         suffix_arrays = []
    #         for text in self.data[column]:
    #             text = str(text)  # Ensure text is a string
    #             suffixes = sorted([text[i:] for i in range(len(text))])
    #             suffix_arrays.append(suffixes)
    #         suffix_dict[f'suffix_array_{column}'] = suffix_arrays
    #     
    #     self.data_blocks = suffix_dict
    #     return self.data_blocks
    
    def tumbling_window(self, window_size):

        if window_size < 1:
            raise ValueError("Window size must be at least 1.")
        
        block_id = 1  # Initialize block counter
        block_ids = []  # To store block IDs for each row
        self.blocks = self.data # Assign raw data to blocks
        
        for i in range(0, len(self.data), window_size):
            window_size_slice = self.data.iloc[i:i + window_size]  # Create a window slice
            # Assign current block_id to all rows in this window
            block_ids.extend([block_id] * len(window_size_slice))
            block_id += 1
        
        self.blocks['block_id'] = block_ids # Add the block_id column to the DataFrame
        self.num_blocks = block_id  # Get the number of blocks
        pass
    
    def build_blocks(self, window_size):
        
        if self.method == 'tumbling_window':
            self.tumbling_window(window_size)
        else:
            raise ValueError("No method has been set. Set a method first.")
        return self.blocks
    
    # Sending parameters and blocks to db
    def send_params_to_db(self):
        # TBD.....
        # STEP OF DATA PIPELINE, BLOCKS - FULL DATA, NUMBER OF BLOCKS, USED METHOD/ALGORITHM, PARAMETERS OF THE PROCES/METHOD/ALGORITHM
        return "block_building", self.blocks, self.num_blocks - 1, f"block_building={self.method}", f"sorted={self.sort_flag}"

In [121]:
bb = BlockBuilding(data) # Loading data for block building, sorting is enabled by default, can be disabled by setting sort_flag=False

# Optionally sort the data
bb.sort_data(columns=['game_name'])

# Apply the tumbling window method with a window size of 2
bb.set_method('tumbling_window')
result_df = bb.build_blocks(window_size=100)
# Send parameters to the database (simulate by printing the return values)
step, blocks, num_blocks, method, sort_flag = bb.send_params_to_db()
print(f"Step: {step}")
# print(f"Blocks:\n{blocks}")
print(f"Number of blocks: {num_blocks}")
print(f"Method: {method}")
print(f"Parameters: {sort_flag}")

result_df

Step: block_building
Number of blocks: 2000
Method: block_building=tumbling_window
Parameters: sorted=True


,user_id,game_name,purchase_play,hours_played,0,block_id
63501,46055854,007 Legends,play,0.7,0,1
63500,46055854,007 Legends,purchase,1.0,0,1
59136,86055705,0RBITALIS,play,0.3,0,1
193301,93030550,0RBITALIS,purchase,1.0,0,1
193302,93030550,0RBITALIS,play,0.3,0,1
...,...,...,...,...,...,...
126158,207424334,theHunter Primal,purchase,1.0,0,2000
40001,62878249,theHunter Primal,play,9.4,0,2000
40000,62878249,theHunter Primal,purchase,1.0,0,2000
173447,159800136,theHunter Primal,play,71.0,0,2000


In [122]:
bb.display_block(3)

,user_id,game_name,purchase_play,hours_played,0,block_id
113479,86256882,3DMark API Overhead feature test,purchase,1.0,0,3
99714,87740709,3DMark API Overhead feature test,purchase,1.0,0,3
105095,45617627,3DMark API Overhead feature test,purchase,1.0,0,3
28216,24469287,3DMark API Overhead feature test,purchase,1.0,0,3
63647,37422528,3DMark API Overhead feature test,purchase,1.0,0,3
...,...,...,...,...,...,...
94062,38465050,404Sight,purchase,1.0,0,3
62509,144580725,404Sight,purchase,1.0,0,3
196966,222180483,404Sight,purchase,1.0,0,3
406,53875128,404Sight,play,0.5,0,3


# Block cleaning
### na razie ustalmy, że to po prostu usuwanie duplikatów, exact albo po kolumnach bo używając tumbling/hopping windows w budowaniu bloków będa bloki takiej samej wielkosci, wiec no ciezko o usuwanie duzych/malych bloków

In [135]:
import pandas as pd

class BlockCleaning:
    
    def __init__(self, blocks):
        self.blocks = blocks
        self.num_blocks = max(blocks['block_id'])
        self.method = None
        self.blocks_cleaned = pd.DataFrame()
        
    def set_method(self, method):
        if method in ['drop_exact_duplicates', 'drop_duplicates_selected_cols']:
            self.method = method
        else:
            raise ValueError("Method not supported")
        
    def drop_exact_duplicates(self):
        self.blocks_cleaned = pd.DataFrame()  # Reset cleaned blocks to empty DataFrame
        
        for block_id, block in self.blocks.groupby('block_id'):
            block_cleaned = block.drop_duplicates()  # Drop exact duplicates
            self.blocks_cleaned = pd.concat([self.blocks_cleaned, block_cleaned])  # Append to blocks_cleaned
        
        return self.blocks_cleaned
    
    def drop_duplicates_selected_cols(self, columns):
        self.blocks_cleaned = pd.DataFrame()  # Reset cleaned blocks to empty DataFrame
        
        for block_id, block in self.blocks.groupby('block_id'):
            block_cleaned = block.drop_duplicates(subset=columns)  # Drop duplicates on specific columns
            self.blocks_cleaned = pd.concat([self.blocks_cleaned, block_cleaned])  # Append to blocks_cleaned
        
        return self.blocks_cleaned
    
    def clean_blocks(self, columns=None):
        if self.method == 'drop_exact_duplicates':
            return self.drop_exact_duplicates()
        
        elif self.method == 'drop_duplicates_selected_cols':
            if columns is None:
                raise ValueError("You must provide columns for 'drop_duplicates_selected_cols' method.")
            return self.drop_duplicates_selected_cols(columns=columns)
        
        else:
            raise ValueError("No valid method set. Set the cleaning method first.")

    def display_clean_block(self, block_index=1):
        if self.blocks_cleaned is None or self.blocks_cleaned.empty:
            raise ValueError("No blocks have been cleaned. Run the blocks_cleaned method first.")
        
        return self.blocks_cleaned[self.blocks_cleaned['block_id'] == block_index]
    
    def get_blocks_cleaned(self):
        if self.blocks_cleaned is None or self.blocks_cleaned.empty:
            raise ValueError("No blocks have been cleaned. Run the blocks_cleaned method first.")
        
        return self.blocks_cleaned


In [136]:
block_cleaner = BlockCleaning(result_df)

block_cleaner.set_method('drop_exact_duplicates')
block_cleaner.clean_blocks()  
block_cleaner.get_blocks_cleaned()

,user_id,game_name,purchase_play,hours_played,0,block_id
63501,46055854,007 Legends,play,0.7,0,1
63500,46055854,007 Legends,purchase,1.0,0,1
59136,86055705,0RBITALIS,play,0.3,0,1
193301,93030550,0RBITALIS,purchase,1.0,0,1
193302,93030550,0RBITALIS,play,0.3,0,1
...,...,...,...,...,...,...
126158,207424334,theHunter Primal,purchase,1.0,0,2000
40001,62878249,theHunter Primal,play,9.4,0,2000
40000,62878249,theHunter Primal,purchase,1.0,0,2000
173447,159800136,theHunter Primal,play,71.0,0,2000


In [137]:
block_cleaner = BlockCleaning(result_df)

block_cleaner.set_method('drop_duplicates_selected_cols')
block_cleaner.clean_blocks(columns=['game_name', 'user_id'])  
block_cleaner.get_blocks_cleaned()

,user_id,game_name,purchase_play,hours_played,0,block_id
63501,46055854,007 Legends,play,0.7,0,1
59136,86055705,0RBITALIS,play,0.3,0,1
193301,93030550,0RBITALIS,purchase,1.0,0,1
112177,11940338,0RBITALIS,play,0.6,0,1
166460,50818751,1... 2... 3... KICK IT! (Drop That Beat Like a...,play,5.0,0,1
...,...,...,...,...,...,...
141839,198914333,theHunter,purchase,1.0,0,2000
62965,157080495,theHunter Primal,purchase,1.0,0,2000
126159,207424334,theHunter Primal,play,4.6,0,2000
40001,62878249,theHunter Primal,play,9.4,0,2000


# Comparison cleaning
### Tu zrobimy porównania oparte o prawdopodobieństwie na danych kolumnach, np. fuzzy string matching, jakieś grafy prawdopodobieństwa, idk